In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [4]:
source_filename = "enron_random_clean1_senders.pkl"
master = pd.read_pickle(source_filename)
master["label"] = master.entity.apply(lambda x: 1 if x == "signature" else 0)

In [5]:
master[["nSigBlocks", "filename"]].groupby("nSigBlocks").agg(["nunique"])

,filename
,nunique
nSigBlocks,
0,35
1,116


In [4]:
master.label.value_counts()

0    2586
1     403
Name: label, dtype: int64

In [5]:
master.label.value_counts(normalize=True)

0    0.865172
1    0.134828
Name: label, dtype: float64

In [6]:
len(master)

2989

In [19]:
master[master.nSig == 1]

,line,filename,entity,len,lineNo,blank,email,url,phone,sigdelimiter,special,words,header,name,endquote,tabs1,tabs2,tabs3,punct20,punct50,punct90,reply,startpunct,firstchar,replypunct,wrote,alphanum90,alphanum50,alphanum10,num90,num50,num10,title,many_titles,person,org,nlines,len_avg,len_min,len_max,nBlanks,nNonBlanks,nSig,nSigBlocks,posFromEnd,last,prevlast,last5,last11,posRatio,posRatioFromEnd,posRatioNB,lenRatio,lenRatioMax,less_avg_len,more_avg_len,less_avg_len75,less_avg_len50,named_entity,pred_file,next_file,pred_named_entity,next_named_entity,prev_same_entity,next_same_entity,sender,sender_name,has_sender,has_sender_name,label,prev_filename,next_filename,prev_label,next_label
148,"Lucy,",allen-p_all_documents_171.txt,no_entity,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,L,0,0,1,0,0,0,0,0,1,1,1,0,5,20.0,0,78,1,4,1,1,4,0,0,1,1,0.20,0.80,0.250000,0.250000,0.064103,1,0,1,1,person,allen-p_all_documents_113.txt,allen-p_all_documents_171.txt,no_entity,no_entity,0,0,phillip.allen@enron.com,phillip allen,False,0,0,allen-p_all_documents_113.txt,allen-p_all_documents_171.txt,0.0,0.0
149,I got your email with the attachment. Let's work together today to get this,allen-p_all_documents_171.txt,no_entity,78,2,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,,0,0,1,0,0,0,0,0,0,0,0,0,5,20.0,0,78,1,4,1,1,3,0,0,1,1,0.40,0.60,0.500000,3.900000,1.000000,0,1,0,0,no_entity,allen-p_all_documents_171.txt,allen-p_all_documents_171.txt,person,no_entity,0,1,phillip.allen@enron.com,phillip allen,False,0,0,allen-p_all_documents_171.txt,allen-p_all_documents_171.txt,0.0,0.0
150,done,allen-p_all_documents_171.txt,no_entity,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,d,0,0,0,0,0,0,0,0,0,0,0,0,5,20.0,0,78,1,4,1,1,2,0,0,1,1,0.60,0.40,0.750000,0.200000,0.051282,1,0,1,1,no_entity,allen-p_all_documents_171.txt,allen-p_all_documents_171.txt,no_entity,blank,1,0,phillip.allen@enron.com,phillip allen,False,0,0,allen-p_all_documents_171.txt,allen-p_all_documents_171.txt,0.0,0.0
151,,allen-p_all_documents_171.txt,no_entity,0,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,0,1,1,1,0,0,0,0,0,0,0,5,20.0,0,78,1,4,1,1,1,0,1,1,1,0.80,0.20,1.000000,0.000000,0.000000,1,0,1,1,blank,allen-p_all_documents_171.txt,allen-p_all_documents_171.txt,no_entity,org,0,0,phillip.allen@enron.com,phillip allen,False,0,0,allen-p_all_documents_171.txt,allen-p_all_documents_171.txt,0.0,1.0
152,Phillip,allen-p_all_documents_171.txt,signature,7,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,P,0,0,0,0,0,0,0,0,1,1,0,1,5,20.0,0,78,1,4,1,1,0,1,0,1,1,1.00,0.00,1.250000,0.350000,0.089744,1,0,1,1,org,allen-p_all_documents_171.txt,allen-p_all_documents_496.txt,blank,person,0,0,phillip.allen@enron.com,phillip allen,False,0,1,allen-p_all_documents_171.txt,allen-p_all_documents_496.txt,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2748,Copyright (C) 2000 Business Wire. All rights reserved.,fossum-d_all_documents_625.txt,signature,54,64,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,C,0,0,1,0,0,0,0,0,0,0,0,0,64,55.0,0,78,8,56,1,1,0,1,0,1,1,1.00,0.00,1.142857,0.981818,0.692308,1,0,0,0,phone,fossum-d_all_documents_625.txt,fossum-d_all_documents_970.txt,blank,url,0,0,lorna.brennan@enron.com,lorna brennan,False,0,1,fossum-d_all_documents_625.txt,fossum-d_all_documents_970.txt,0.0,0.0
2972,The call tomorrow is to discuss legal strucutre of the ECA and is not a,gay-r_all_documents_46.txt,no_entity,72,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,T,0,0,1,0,0,0,0,0,0,0,0,0,4,40.0,0,75,1,3,1,1,3,0,0,1,1,0.25,0.75,0.333333,1.800000,0.960000,0,1,0,0,no_entity,gay-r_all_documents_383.txt,gay-r_all_documents_46.txt,no_entity,no_entity,1,1,rob.gay@enron.com,rob gay,False,0,0,gay-r_all_documents_383.txt,gay-r_all_documents_46.txt,0.0,0.0
2973,numbers call. We will need plenty of number soon though so please proceed.,

# What percentage of signature lines are preceded by another signature?

In [13]:
master["prev_filename"] = master.filename.shift(1)
master["next_filename"] = master.filename.shift(-1)

master["prev_label"] = master.label.shift(1)
master["prev_label"] = master.apply(lambda row: row.prev_label if row.filename == row.prev_filename else np.NaN, axis=1)
master["prev_label"] = master["prev_label"].fillna(value=0)

master["next_label"] = master.label.shift(-1)
master["next_label"] = master.apply(lambda row: row.next_label if row.filename == row.next_filename else np.NaN, axis=1)
master["next_label"] = master["next_label"].fillna(value=0)

In [108]:
# master.loc[150:200, ["prev_filename", "filename", "prev_label", "label"]]
# master.loc[150:200, ["filename", "next_filename", "label", "next_label"]]

In [115]:
master[(master.label == 0) & (master.prev_label == 1)]

,line,filename,entity,len,lineNo,blank,email,url,phone,sigdelimiter,special,words,header,name,endquote,tabs1,tabs2,tabs3,punct20,punct50,punct90,reply,startpunct,firstchar,replypunct,wrote,alphanum90,alphanum50,alphanum10,num90,num50,num10,title,many_titles,person,org,nlines,len_avg,len_min,len_max,nBlanks,nNonBlanks,nSig,nSigBlocks,posFromEnd,last,prevlast,last5,last11,posRatio,posRatioFromEnd,posRatioNB,lenRatio,lenRatioMax,less_avg_len,more_avg_len,less_avg_len75,less_avg_len50,named_entity,pred_file,next_file,pred_named_entity,next_named_entity,prev_same_entity,next_same_entity,sender,sender_name,has_sender,has_sender_name,label,prev_label,next_label,prev_filename,next_filename
744,,baughman-e_all_documents_255.txt,no_entity,0,21,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,0,1,1,1,0,0,0,0,0,0,0,68,38.0,0,77,13,55,3,1,47,0,0,0,0,0.308824,0.691176,0.381818,0.0,0.0,1,0,1,1,blank,baughman-e_all_documents_255.txt,baughman-e_all_documents_255.txt,no_entity,no_entity,0,0,kayne.coulter@enron.com,kayne coulter,False,0,0,1.0,0.0,baughman-e_all_documents_255.txt,baughman-e_all_documents_255.txt
1086,,campbell-l_all_documents_1167.txt,no_entity,0,12,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,0,1,1,1,0,0,0,0,0,0,0,28,36.0,0,66,4,24,4,1,16,0,0,0,0,0.428571,0.571429,0.500000,0.0,0.0,1,0,1,1,blank,campbell-l_all_documents_1167.txt,campbell-l_all_documents_1167.txt,phone,no_entity,0,0,owner-nyiso_tech_exchange@lists.thebiz.net,owner nyiso tech exchange,False,0,0,1.0,0.0,campbell-l_all_documents_1167.txt,campbell-l_all_documents_1167.txt
2343,,fischer-m_all_documents_126.txt,no_entity,0,7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,0,1,1,1,0,0,0,0,0,0,0,19,24.0,0,76,7,12,1,1,12,0,0,0,0,0.368421,0.631579,0.583333,0.0,0.0,1,0,1,1,blank,fischer-m_all_documents_126.txt,fischer-m_all_documents_126.txt,person,blank,0,1,matthew.meyers@enron.com,matthew meyers,False,0,0,1.0,0.0,fischer-m_all_documents_126.txt,fischer-m_all_documents_126.txt
2777,,fossum-d_all_documents_970.txt,no_entity,0,29,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,0,0,1,1,1,0,0,0,0,0,0,0,219,58.0,0,78,20,199,6,1,190,0,0,0,0,0.132420,0.867580,0.145729,0.0,0.0,1,0,1,1,blank,fossum-d_all_documents_970.txt,fossum-d_all_documents_970.txt,url,no_entity,0,0,issuealert@scientech.com,issuealert,False,0,0,1.0,0.0,fossum-d_all_documents_970.txt,fossum-d_all_documents_970.txt


In [14]:
master[master.label == 1].next_label.value_counts(normalize=False, dropna=False, sort=False)

1.0    287
0.0    116
Name: next_label, dtype: int64

In [134]:
master[master.label == 1].prev_label.value_counts(normalize=False, dropna=False, sort=False)

0.0    116
1.0    287
Name: prev_label, dtype: int64

In [130]:
print(f"{0.955916*100:.1f}")
print(f"{0.044084*100:.1f}")

95.6
4.4


In [38]:
master.label.value_counts()

0    2586
1    403 
Name: label, dtype: int64

In [42]:
master[master.label == 1].prev_label.unique()

array([ 1.,  0., nan])

In [ ]:
master[master.label == 1].

In [7]:
master[["label", "prev_label", "next_label"]].groupby("label").agg(["sum"])

,prev_label,next_label
,sum,sum
label,,
0,4.0,114.0
1,287.0,287.0


In [32]:
master[["nSigBlocks", "filename"]].groupby("nSigBlocks").nunique()

,nSigBlocks,filename
nSigBlocks,,
0,1,35
1,1,116


In [36]:
len(master[master.nSigBlocks == 0].filename.unique())

35

In [21]:
master[(master.next_label == 1) & (master.prev_label == 1)].label.value_counts(normalize=False, dropna=False, sort=False)

1    211
Name: label, dtype: int64